In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt 
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation
from sklearn.metrics import classification_report, confusion_matrix
import spacy
import re

In [ ]:
#SET random seed
np.random.seed(1997)

In [ ]:
#Read the training & Test dataset using Pandas

training = pd.read_csv("hasoc_train.csv")
test = pd.read_csv("hasoc_test.csv")

In [ ]:
#Recode labels for Training task_1 to numeric
cleanup_nums = {"task_1": {"HOF": 1, "NOT": 0}}
training = training.replace(cleanup_nums)
training.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,0,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,1,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,0,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,0,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,0,NONE,NONE


In [ ]:
#Recode labels for Test SubtaskA to numeric
cleanup_nums2 = {"task_1": {"HOF": 1, "NOT": 0}}
test = test.replace(cleanup_nums2)
test.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...,0,NONE,NONE
1,hasoc_en_416,68.5 million people have been forced to leave ...,0,NONE,NONE
2,hasoc_en_207,"You came, you saw .... we will look after the ...",0,NONE,NONE
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...,0,NONE,NONE
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...,1,PRFN,UNT


In [ ]:
#Preview some Test tweets before processing
a = 0 
for i in range(a,a+10):
    print(test["text"][i])
    print()

West Bengal Doctor Crisis: Protesting doctors agree to meet Mamata Banerjee in presence of full media even as IMA goes on strike  

68.5 million people have been forced to leave their homes.      Read more: https://wef.ch/2YQcwpk  #refugees #society

You came, you saw .... we will look after the fort! Good luck! 

We'll get Brexit delivered by October 31st.    Help build the movement that will do it  http://conservatives.com/join     #BackBoris @BorisJohnson

Fuck you. Go back to the dark ages you cow @IBNLiveRealtime: Rapes happen because men and women interact freely: Mamata

Boris Johnson faces Supreme Court bid to make him stand trial for ‘lying and misleading’ in Brexit campaign https://www.independent.co.uk/news/uk/crime/boris-johnson-brexit-supreme-court-vote-leave-nhs-bus-trial-appeal-a9029506.html …

What about a refund for not serving Halala to Muslims and regularly adding Onions to Jain food !! Does Food not carries a religion here ??

General election, DUP dumped out, Tory 

In [ ]:
def words(text, stem_words=False):
    #Remove emojis 
    emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
    "]+", flags=re.UNICODE)
    
    text = emoji_pattern.sub(r'', text) # No emoji
    #text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) # Remove urls
    text = re.sub(r'http\S+', '', text) # Remove urls
    # Lowering the words in Tweets
    text = text.lower()
    # Remove punctuation from Tweets
    text = ''.join([c for c in text if c not in punctuation])
    #Use filter and lambda to remove numeric digits from string
    res = "".join(filter(lambda x: not x.isdigit(), text))
    text = str(res)
    #remove leading and ending spaces
    text = text.strip()
    # Return a list of words
    return(text)

def process(tweet_list, tweets):
    for tweet in tweets:
        tweet_list.append(words(tweet))

def tokenize(tweet_list, tweets):
    for tweet in tweets:
        tokens = re.findall("[\w]+", tweet)
        tweet_list.append(tokens)

In [ ]:
#Processing training and test set
processed_tweet1 = []
processed_tweet2 = []
process(processed_tweet1, training["text"])
process(processed_tweet2, test["text"])

#Preview some test tweets after processing
a = 0 
for i in range(a,a+10):
    print(processed_tweet2[i])
    print()

west bengal doctor crisis protesting doctors agree to meet mamata banerjee in presence of full media even as ima goes on strike

million people have been forced to leave their homes      read more   refugees society

you came you saw  we will look after the fort good luck

well get brexit delivered by october st    help build the movement that will do it       backboris borisjohnson

fuck you go back to the dark ages you cow ibnliverealtime rapes happen because men and women interact freely mamata

boris johnson faces supreme court bid to make him stand trial for ‘lying and misleading’ in brexit campaign  …

what about a refund for not serving halala to muslims and regularly adding onions to jain food  does food not carries a religion here

general election dup dumped out tory power weakened the only way out

repost freewicked  • • • • • •  freewicked freethekids terrorist americanterrorist fucktrump donaldtrump fuckdonaldtrump notothewall nowall ak shooting justiceforthekids nomoregun

In [ ]:
tokenized_tweet1 = []
tokenized_tweet2 = []
tokenize(tokenized_tweet1, processed_tweet1)
tokenize(tokenized_tweet2, processed_tweet2)

#Preview some test tweets after tokenization
a = 0 
for i in range(a,a+10):
    print(tokenized_tweet2[i])
    print()

['west', 'bengal', 'doctor', 'crisis', 'protesting', 'doctors', 'agree', 'to', 'meet', 'mamata', 'banerjee', 'in', 'presence', 'of', 'full', 'media', 'even', 'as', 'ima', 'goes', 'on', 'strike']

['million', 'people', 'have', 'been', 'forced', 'to', 'leave', 'their', 'homes', 'read', 'more', 'refugees', 'society']

['you', 'came', 'you', 'saw', 'we', 'will', 'look', 'after', 'the', 'fort', 'good', 'luck']

['well', 'get', 'brexit', 'delivered', 'by', 'october', 'st', 'help', 'build', 'the', 'movement', 'that', 'will', 'do', 'it', 'backboris', 'borisjohnson']

['fuck', 'you', 'go', 'back', 'to', 'the', 'dark', 'ages', 'you', 'cow', 'ibnliverealtime', 'rapes', 'happen', 'because', 'men', 'and', 'women', 'interact', 'freely', 'mamata']

['boris', 'johnson', 'faces', 'supreme', 'court', 'bid', 'to', 'make', 'him', 'stand', 'trial', 'for', 'lying', 'and', 'misleading', 'in', 'brexit', 'campaign']

['what', 'about', 'a', 'refund', 'for', 'not', 'serving', 'halala', 'to', 'muslims', 'and', 'r

# TFIDF Vectorizer

In [ ]:
def dummy_fun(tokenized_tweet1):
    return tokenized_tweet1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer = 'word',
                        tokenizer=dummy_fun,
                        preprocessor=dummy_fun,
                        lowercase = True,
                        max_features = 300,
                        strip_accents='unicode',
                        norm = 'l2')

In [ ]:
#training set
tfidf1=tfidf.fit(tokenized_tweet1)
transformed_tweets = tfidf1.transform(tokenized_tweet1)

#test set 
transformed_tweets2 = tfidf.transform(tokenized_tweet2)

#Assigning independent and dependent variables for training set
x = transformed_tweets
y = training['task_1']

#Assigning independent and dependent variables for test set
x1 = transformed_tweets2
y1 = test['task_1']

#Organizing variables into train and test set
x_train = x
y_train = y
x_test = x1
y_test = y1

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


# SVM (Linear SVM)

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC(tol=1e-05)
svm.fit(x_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=1e-05,
          verbose=0)

In [ ]:
#Predicting the test results
y_pred = svm.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.41      0.43      0.42       288
           0       0.81      0.79      0.80       865

    accuracy                           0.70      1153
   macro avg       0.61      0.61      0.61      1153
weighted avg       0.71      0.70      0.71      1153



# CNN

In [ ]:
from pandas import DataFrame
#training set
df0 = DataFrame(processed_tweet1,columns=['text'])
#test set
df00 = DataFrame(processed_tweet2,columns=['text'])

In [ ]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Sequential

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])
 
# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded
 
# define the model
def define_model(length, vocab_size):
    # channel 1
    model = Sequential()
    model.add(Embedding(vocab_size, 100,input_shape=(length,)))
    model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    # interpretation
    model.add(Dense(1, activation='sigmoid')) 
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    return model

In [ ]:
from sklearn.model_selection import train_test_split

# Organize  dataset
x_train = df0['text']
y_train = training['task_1']
x_test = df00['text']
y_test = test['task_1']

# create tokenizer
tokenizer = create_tokenizer(x_train)
# calculate max document length
length = max_length(x_train)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, x_train, length)
testX = encode_text(tokenizer, x_test, length)

print(trainX.shape,testX.shape)
 
# define model
model = define_model(length, vocab_size)

Max document length: 92
Vocabulary size: 17949
(5852, 92) (1153, 92)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 92, 100)           1794900   
_________________________________________________________________
conv1d (Conv1D)              (None, 89, 32)            12832     
_________________________________________________________________
dropout (Dropout)            (None, 89, 32)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 44, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1408)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1409      
Total params: 1,809,141
Trainable params: 1,809,141
N

In [ ]:
# fit model
results = model.fit(trainX, array(y_train), epochs=10, batch_size=32, verbose=1,validation_data=(testX, array(y_test)))

Epoch 1/10
183/183 [==============================] - 23s 37ms/step - loss: 0.6687 - accuracy: 0.6145 - val_loss: 0.5825 - val_accuracy: 0.7502
Epoch 2/10
183/183 [==============================] - 6s 35ms/step - loss: 0.5849 - accuracy: 0.6781 - val_loss: 0.5533 - val_accuracy: 0.7493
Epoch 3/10
183/183 [==============================] - 6s 35ms/step - loss: 0.3032 - accuracy: 0.8985 - val_loss: 0.5933 - val_accuracy: 0.7112
Epoch 4/10
183/183 [==============================] - 6s 34ms/step - loss: 0.1195 - accuracy: 0.9671 - val_loss: 0.6377 - val_accuracy: 0.6869
Epoch 5/10
183/183 [==============================] - 6s 34ms/step - loss: 0.0666 - accuracy: 0.9819 - val_loss: 0.6821 - val_accuracy: 0.6973
Epoch 6/10
183/183 [==============================] - 6s 33ms/step - loss: 0.0527 - accuracy: 0.9836 - val_loss: 0.6885 - val_accuracy: 0.6973
Epoch 7/10
183/183 [==============================] - 6s 34ms/step - loss: 0.0379 - accuracy: 0.9874 - val_loss: 0.7395 - val_accuracy: 0.687

In [ ]:
pred = model.predict_classes(testX)
print(classification_report(y_test,pred,labels=[1,0]))

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           1       0.41      0.62      0.49       288
           0       0.85      0.70      0.77       865

    accuracy                           0.68      1153
   macro avg       0.63      0.66      0.63      1153
weighted avg       0.74      0.68      0.70      1153

